In [1]:
import pandas as pd
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [2]:
from s4_reg.core import s4regressor as regressor
from s4_reg.src_dataloaders_original import StandardScaler
from s4_reg.src_utils_visualize import prediction_result as post

c:\Users\rodin\work\stock_trade\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\rodin\work\stock_trade\.venv\lib\site-packages\torchaudio\backend\utils.py:74: UserWarning: No audio backend is available.
  warnings.warn("No audio backend is available.")


In [3]:
import pickle

with open("tests/prepared_data.pkl", "rb") as tf:
    dict_prepared_data = pickle.load(tf)

def make_s4_features(
    data,
    target,
    seq_len_target=180,
    pred_len_target=1,
    d_model_target=10, # 2048
    seq_len_others=10,
    pred_len_others=1,
    d_model_others=10
    ):
    
    data = data.reset_index()
    data['date'] = pd.to_datetime(data['Date'])
    data = data.drop(['Date'],axis=1)

    assert seq_len_target > seq_len_others
       
    model_target = regressor(
        dataset = data,
        target = target,
        size = [seq_len_target, pred_len_target],
        features = 'S',
        d_model = d_model_target,
        device = 'cpu'
    )
    
    feat_df_target = model_target.get_features(data)
    stock_data = feat_df_target[target]
    feat_df_target = feat_df_target.drop([target], axis=1)
    
    model_others = regressor(
        dataset = data,
        target = target,
        size = [seq_len_others, pred_len_others],
        features = 'MS',
        d_model = d_model_others,
        device = 'cpu'
    )
    
    feat_df_others = model_others.get_features(data).iloc[seq_len_target-seq_len_others:,:]
    feat_df_others = feat_df_others.drop([target], axis=1)
    feat_df_others.columns = [f'exog_feat_{i+1}' for i in range(len(feat_df_others.columns))]

    features = pd.concat([
                          feat_df_target,
                          feat_df_others
                          ], axis=1)
    
    return features.iloc[:-1,:], pd.DataFrame(stock_data.iloc[:-1])

targets = [
    '4584.T',
    '1557.T',
    '8789.T',
    '1893.T',
    'MSFT'
]

features = {}
for i, target in enumerate(targets):
    feat, stock = make_s4_features(dict_prepared_data[target], target)
    if i==0:
        features[target] = feat
        stock_data = stock
    else:
        features[target] = feat
        stock_data = pd.concat([stock_data, stock], axis=1)   
    

CUDA extension for Cauchy multiplication not found. Install by going to extensions/cauchy/ and running `python setup.py install`. This should speed up end-to-end training by 10-50%
Falling back on slow Cauchy kernel. Install at least one of pykeops or the CUDA extension for memory efficiency.
Falling back on slow Vandermonde kernel. Install pykeops for improved memory efficiency.


In [4]:
display(features)
display(stock_data)

{'4584.T':           feat_1      feat_2     feat_3      feat_4      feat_5      feat_6  \
 180   759.772217  124.378746  35.910980 -355.077606  450.786469 -466.541962   
 181   780.926331  130.576157  35.047318 -363.625000  462.847321 -479.167175   
 182   785.210571  130.537582  36.907253 -366.455231  465.384583 -481.690155   
 183   724.858215  121.349426  31.804905 -340.099091  427.807434 -443.279755   
 184   715.975830  118.337715  32.622684 -333.855835  420.758728 -437.826019   
 ...          ...         ...        ...         ...         ...         ...   
 1248  240.497849   41.462242  13.131885 -109.656715  145.300552 -148.154724   
 1249  237.722244   43.641411  10.955653 -108.320877  143.320831 -149.069626   
 1250  244.596085   42.466618  12.683497 -111.768013  145.887741 -148.545349   
 1251  240.614731   40.725067  10.429011 -111.842094  145.893997 -151.098907   
 1252  238.514633   42.008884  13.450680 -108.465218  145.272293 -147.449829   
 
           feat_7      feat_

,4584.T,1557.T,8789.T,1893.T,MSFT
180,816.0,28420.0,149.0,537.042297,101.410782
181,822.0,28830.0,147.0,549.849304,102.930206
182,758.0,29120.0,137.0,547.287903,100.990318
183,749.0,28960.0,138.0,540.457520,101.974594
184,808.0,28870.0,138.0,532.773193,101.487228
...,...,...,...,...,...
1248,249.0,55370.0,65.0,631.000000,288.799988
1249,254.0,55640.0,67.0,635.000000,288.369995
1250,253.0,55630.0,67.0,629.000000,288.450012
1251,251.0,55640.0,67.0,628.000000,286.109985


In [5]:
from ray.tune import register_env

# 環境クラスのインポート
from gym_stock_trading_env import StockTradingEnv

# Define a function to create the environment
def create_stock_trading_env(_):
    env = StockTradingEnv(stock_data, features)
    return env

# Register the environment
env_name = "stock_trading_env"
register_env(env_name, create_stock_trading_env)

In [7]:
# import gymnasium as gym
import ray
from ray.rllib.algorithms.ppo import PPOConfig
from ray.rllib.algorithms.dqn import DQNConfig

# PPOConfig.environment(disable_env_checking=True)
# DQNConfig.environment(disable_env_checking=True)

# Initialize Ray in local mode
ray.init(ignore_reinit_error=True, local_mode=True)

# Create the environment
env = StockTradingEnv(stock_data, features)

# Create PPO agent
ppo_config = PPOConfig().environment(env=env_name)
ppo_agent = ppo_config.build()

# Train the agents
num_iterations = 10

for i in range(num_iterations):
    ppo_result = ppo_agent.train()
    # dqn_result = dqn_agent.train()
    print(f"Iteration {i + 1}:")
    print(f"PPO: episode_reward_mean={ppo_result['episode_reward_mean']}")

# Evaluate the trained agents
def evaluate(agent, env):
    episode_reward = 0
    terminated = False
    obs = env.reset()

    while not terminated:
        action = agent.compute_single_action(obs)
        obs, reward, terminated, _ = env.step(action)
        episode_reward += reward

    return episode_reward

print("\nEvaluating trained agents:")
print(f"PPO: {evaluate(ppo_agent, env)}")

# Shutdown Ray
ray.shutdown()


2023-05-09 01:04:38,879	INFO worker.py:1454 -- Calling ray.init() again after it has already been called.
:actor_name:RolloutWorker
:actor_name:RolloutWorker
2023-05-09 01:04:38,987	WARNING util.py:67 -- Install gputil for GPU system monitoring.


:actor_name:RolloutWorker
:actor_name:RolloutWorker
Iteration 1:
PPO: episode_reward_mean=26740534.956261754
Iteration 2:
PPO: episode_reward_mean=106779938.77561693
Iteration 3:
PPO: episode_reward_mean=133604222.52575299
Iteration 4:
PPO: episode_reward_mean=140973899.9382567
Iteration 5:
PPO: episode_reward_mean=165548033.00363925
Iteration 6:
PPO: episode_reward_mean=173857002.68560383
Iteration 7:
PPO: episode_reward_mean=182374462.7843407
Iteration 8:
PPO: episode_reward_mean=186907041.44266376
Iteration 9:
PPO: episode_reward_mean=193746350.60624272
Iteration 10:
PPO: episode_reward_mean=202322647.4261042

Evaluating trained agents:


ValueError: The two structures don't have the same nested structure.

First structure: type=tuple str=({'T_4584_T': array([  816.      ,   759.7722  ,   124.378746,    35.91098 ,
        -355.0776  ,   450.78647 ,  -466.54196 ,  -209.51523 ,
        -668.83704 ,  -765.913   ,   446.53607 , -3296.4219  ,
         -81.178085,  -725.35504 ,   409.0702  ,  3429.948   ,
        1671.3894  ,  1932.4185  , -2117.947   , -1082.8972  ,
        -933.7444  ], dtype=float32), 'T_1557_T': array([ 28420.     ,  27069.906  ,   4526.612  ,   1195.2346 ,
       -12677.845  ,  15991.0625 , -16409.21   ,  -7438.825  ,
       -23656.59   , -27206.695  ,  15819.948  ,  -3414.8308 ,
          743.98334,   -746.6085 ,   -212.78941,   3106.8596 ,
         1120.533  ,   2602.14   ,  -1764.2444 ,    -54.14319,
        -1516.7076 ], dtype=float32), 'T_8789_T': array([  149.      ,   141.81313 ,    20.981348,     8.681377,
         -65.64669 ,    85.784325,   -92.08866 ,   -39.716877,
        -128.91243 ,  -144.8